### Pipeline

1) Generating images
2) Generating text images with empty background
3) Combining images with texts using replicate
4) Combining new images into a video using shot stack


In [1]:
from PIL import Image, ImageDraw, ImageFont
import requests
import json
import replicate
import os
import time
import hashlib

In [12]:
# Images
input_path = "../resources/"
image_filenames = [input_path + "3_horses.jpg", input_path + "3_horses_2.jpeg", input_path + "cat_in_sweater.png", input_path + "christmas_tree.jpg"]

In [13]:
# Texts

def generate_text(text: str, 
                  font_name: str = '../resources/morninggloryandcyrillic.otf', 
                  backgroud_path: str = '../resources/empty_background.jpg',
                  output_dir: str = '../image_outputs',
                  font_size: int = 112, 
                  text_color: tuple[int, int, int] = (200, 29, 156),
                  ) -> str:
    """
    Generate specified text with empty background. 
    font_name is the name of a font file located in the working directory or in C\\Windows\\Fonts.
    """
    # empty background
    image = Image.open(backgroud_path)
    image = image.convert('RGB')

    # Initialize drawing context
    draw = ImageDraw.Draw(image)

    # Configure text properties
    font = ImageFont.truetype(font_name, font_size)  # load Cyrillic-capable font

    # Calculate text position (centered)
    text_bbox = draw.textbbox((0, 0), text, font=font)
    text_width = text_bbox[2] - text_bbox[0]
    text_height = text_bbox[3] - text_bbox[1]
    x = (image.width - text_width) / 2
    y = (image.height - text_height) / 2

    # Draw text on image
    draw.text((x, y), text, fill=text_color, font=font)

    # Save the image
    time_str = str(time.time_ns())
    file_name = hashlib.md5(time_str.encode()).hexdigest()
    file_path = os.path.join(output_dir, str(file_name) + '.png')
    image.save(file_path)

    return file_path

In [14]:
# Making empty images with text
texts = ['С Новым годом!', "Пусть ваши мечты сбываются", "В год кота к вам\nобязательно прибудет счастье", "Увидимся в 2026!"]
text_filenames = [generate_text(t, font_size=int(120 - 1.2*len(t))) for t in texts]


In [15]:
COMBINATION_PROMPT = "Add an inscription from the second image to the top or to the bottom of the first image. Do not add any extra text " \
"that is not on my images. You can move the text freely to the best location. The result should look like a card my grandmother might send me."


def add_text_to_image(text_image_path: str, 
                      main_image_path: str, 
                      output_dir: str = '../image_outputs',
                      output_size: str = '1K', 
                      aspect_ratio: str = '4:3',
                      save_locally: bool = True) -> str:
    """
    Combine one main image with one text image.
    Must have REPLICATE_API_TOKEN=... in the .env
    """
    #TODO: make async

    # Upload images to replicate to be able to use them
    with open(text_image_path, "rb") as file:
        text_image = replicate.files.create(file)
    with open(main_image_path, 'rb') as file:
        generated_image = replicate.files.create(file)


    input = {
        "size": output_size,
        "prompt": COMBINATION_PROMPT, 
        "aspect_ratio": aspect_ratio,
        "image_input": [generated_image.urls['get'], text_image.urls['get']]
    }

    output = replicate.run(
        "bytedance/seedream-4",
        input=input
    )

    # To write the files to disk:
    if save_locally:
        for index, item in enumerate(output):
            time_str = str(time.time_ns())
            file_name = hashlib.md5(time_str.encode()).hexdigest()
            with open(os.path.join(output_dir, str(file_name) + '.jpg'), "wb") as file:
                file.write(item.read())
            
    return str(output[0])



In [16]:
# Will run for a couple of minutes
images_with_text = [add_text_to_image(text_filenames[i], image_filenames[i]) for i in range(len(text_filenames))]

In [17]:
images_with_text

['https://replicate.delivery/xezq/84T4kLMzec3PcCP634DFdaqseBOH3BViJMfzqSfkzRm0bAzVB/tmpt48oksj1.jpg',
 'https://replicate.delivery/xezq/kimVbGRzidKaH1c3Xh6AxDkM1dZjW0QDVhCgiCmZ6VepDYuKA/tmpj7womds5.jpg',
 'https://replicate.delivery/xezq/chDiVJbKeQSEMSFe4IaeeHWyAKclwZXOeQ2LpNo702Cm9AmrC/tmpx0rq582q.jpg',
 'https://replicate.delivery/xezq/e1XHxROiewrFQ0Kj7OznK18qMz4QTOS08eWC2sSH1hWIQg5qA/tmpob44khff.jpg']

In [ ]:
# Combining images into a video
API_KEY = os.environ['SHOT_STACK_API_TOKEN']

# for production use "https://api.shotstack.io/v1/render"
url = "https://api.shotstack.io/stage/render"
headers = {
    "x-api-key": API_KEY,
    "Content-Type": "application/json"
}

payload = {
    "timeline": {
        "tracks": [
            {
                "clips": [
                    {
                        "asset": {"type": "image", "src": images_with_text[0]},
                        "start": 0,
                        "length": 3,
                        "transition": {"out": "slideRight"}
                    },
                    {
                        "asset": {"type": "image", "src": images_with_text[1]},
                        "start": 3,
                        "length": 3,
                        "transition": {"in": "wipeLeft", "out": "fade"}
                    },
                    {
                        "asset": {"type": "image", "src": images_with_text[2]},
                        "start": 6,
                        "length": 3,
                        "transition": {"in": "slideUp", "out": "slideDown"}
                    },
                    {
                        "asset": {"type": "image", "src": images_with_text[3]},
                        "start": 9,
                        "length": 3,
                        "transition": {"in": "fade"}
                    }
                ]
            }
        ]
    },
    "output": {"format": "mp4", "resolution": "hd", "aspectRatio": "4:3"}
}

response = requests.post(url, headers=headers, data=json.dumps(payload))
print(response.status_code, response.text)


201 {"success":true,"message":"Created","response":{"message":"Render Successfully Queued","id":"8ec53e29-1721-4685-88cc-c0afcf2df688"}}


In [ ]:
# Wait a bit before running
RENDER_ID = json.loads(response.text)['response']['id']

# for production use f"https://api.shotstack.io/v1/render/{RENDER_ID}"
url = "https://api.shotstack.io/stage/render"
r = requests.get(url + f"/{RENDER_ID}", headers=headers)

print(r.json())
print("Video url:", r.json()['response']['url'])

{'success': True, 'message': 'OK', 'response': {'id': '8ec53e29-1721-4685-88cc-c0afcf2df688', 'owner': '1mj5p7ly7k', 'plan': 'freeTrial', 'status': 'done', 'error': '', 'duration': 12, 'billable': 12, 'renderTime': 3708.53, 'url': 'https://shotstack-api-stage-output.s3-ap-southeast-2.amazonaws.com/1mj5p7ly7k/8ec53e29-1721-4685-88cc-c0afcf2df688.mp4', 'poster': None, 'thumbnail': None, 'data': {'output': {'format': 'mp4', 'resolution': 'hd', 'aspectRatio': '4:3'}, 'timeline': {'tracks': [{'clips': [{'start': 0, 'length': 3, 'asset': {'type': 'image', 'src': 'https://replicate.delivery/xezq/84T4kLMzec3PcCP634DFdaqseBOH3BViJMfzqSfkzRm0bAzVB/tmpt48oksj1.jpg'}, 'transition': {'out': 'slideRight'}}, {'start': 3, 'length': 3, 'asset': {'type': 'image', 'src': 'https://replicate.delivery/xezq/kimVbGRzidKaH1c3Xh6AxDkM1dZjW0QDVhCgiCmZ6VepDYuKA/tmpj7womds5.jpg'}, 'transition': {'in': 'wipeLeft', 'out': 'fade'}}, {'start': 6, 'length': 3, 'asset': {'type': 'image', 'src': 'https://replicate.delive